In [ ]:
!pip install -r requirements.txt

Wir speichern den API Key ab und laden ein Modell.

In [1]:
#mit Secret Manager in Google Colab
from google.colab import userdata
OPENAI_API_KEY = userdata.get('secret_name')

#Youtube-Video
#"https://www.youtube.com/watch?v=cdiD-9MMpb0"

In [6]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

Wir stellen jetzt dem Modell eine simple Frage.

In [7]:
model.invoke("Was ist die Hauptstadt von Deutschland?")

AIMessage(content='Die Hauptstadt von Deutschland ist Berlin.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'stop', 'logprobs': None}, id='run-f46912c8-4e46-49ee-870b-77fe0b7dc655-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Der Rückgabewert ist ein AIMessage-Objekt. Mit einem Output Parser können wir die Antwort daraus extrahieren.

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Was ist die Hauptstadt von Deutschland?")

'Die Hauptstadt von Berlin ist Berlin selbst. Berlin ist eine Stadt und ein Bundesland in Deutschland und hat somit den Status einer Hauptstadt.'

Wir wollen jetzt dem Modell einen Kontext und eine Frage übergeben.

In [9]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Ralfs Bruder heißt Axl", question="Wie heißt Ralfs Bruder?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Ralfs Bruder heißt Axl\n\nQuestion: Wie heißt Ralfs Bruder?\n'

Wir nutzen jetzt langchain, um dem Prompt mit dem Modell und dem Parser zu verketten, d.h. der Output des Prompts ist Input des Modells, und der Output des Modells ist Input des Parsers.

In [10]:
chain = prompt | model | parser
chain.invoke({
    "context": "Ralfs Bruder heißt Axl.",
    "question": "Wie heißt Ralfs Bruder?"
})

'Ralfs Bruder heißt Axl.'

Wir können sogar Ketten kombinieren, um komplexere Workflows zu bauen. Beispielsweise können wir eine zweite Kette definieren, die die Antwort der ersten Kette in eine andere Sprache übersetzt.

In [11]:
translation_prompt = ChatPromptTemplate.from_template(
    "Übersetze {answer} in {language}"
)

Wir können jetzt eine neue Übersetzungs-Kette bauen die das Ergebnis der ersten Kette kombiniert mit dem Übersetzungs-Prompt.

In [12]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Ralfs Bruder heißt Axl. Er hat zwei weitere Geschwister.",
        "question": "Wie viele Geschwister hat Ralf?",
        "language": "Englisch",
    }
)

'Ralf has a total of three siblings.'

Wir lesen die Transkription des o.g. youtube-Videos ein, und zeigen die ersten 100 Zeichen.

In [15]:
with open("interview.txt") as file:
  interview = file.read()

interview[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

WIr können jetzt dem Modell das gesamte Interview als Kontext übergeben. Allerdings hängt es vom Modell und dessen erlaubter context size ab, ob es auch funktioniert.

In [14]:
chain.invoke({
    "context": transcription,
    "question": "Is reading papers a good idea?"
    })

"Yes, reading papers is a good idea, especially in the field of AI and machine learning. It allows you to stay updated on the latest research, understand new methodologies, and gain insights into the current state of the field. Papers often provide detailed explanations and experimental results that can enhance your knowledge and inform your work. However, it's important to recognize that the practical application and implementation of ideas in real-world scenarios are also crucial, and it's beneficial to complement reading papers with hands-on experimentation and coding."

Selbst wenn das Modell den kompletten Kontext aufnehmen kann, macht es Sinn, diesen zu splitten, und nur den relevanten Kontext mitzuliefern. Wir bilden also Chunks der Dokumente und wollen dem Modell anschließend nur die relevanten Chunks mitgeben.

Dazu laden wir zunächst aus Vereinfachungsgründen das gesamte Interview in den Speicher:

In [22]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("interview.txt")
text_documents = loader.load()

Es gibt viele verschiedene Möglichkeiten, ein Dokuments in Chunks zu splitten. Für dieses Beispiel nutzen wir einen simplen Splitter, der die Dokumente in Chunks mit einer fixierten Größe aufteilt.

In diesem Beispiel splitten wir in Chunks mit einer Größe von 100 Zeichen und einem Overlap von 20 Zeichen, um den Kontext zwischen Sätzen nicht zu verlieren.

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(metadata={'source': 'interview.txt'}, page_content="I think it's possible that physics has exploits and we should be trying to find them. arranging some"),
 Document(metadata={'source': 'interview.txt'}, page_content='arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow,'),
 Document(metadata={'source': 'interview.txt'}, page_content='buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences'),
 Document(metadata={'source': 'interview.txt'}, page_content="intelligences are kind of like the next stage of development. And I don't know where it leads to."),
 Document(metadata={'source': 'interview.txt'}, page_content='where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These')]

Für unseren speziellen Anwendungsfall verwenden wir 1000:

In [54]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

Gegeben eine bestimmte Frage, müssen wir die relevanten Chunks aus dem Interview finden, die wir dann an das Modell senden. Hier kommt das Konzept der Embeddings ins Spiel.

Ein Embedding ist eine mathematisch Repräsentation der semantischen Bedeutung eines Wortes, Sätzes oder Dokumentes. Es ist quasi eine Projektion eines Konzeptes in einem höherdimensionalen Raum. Embeddings haben eine simple Eigenschaft: Die Projektion von verwandten Konzepten sind nahe beieinander, während Konzepte mit unterschiedlichen Bedeutungen weiter voneinander entfernt liegen. Siehe auch https://dashboard.cohere.com/playground/embed, um Embeddings in zwei Dimensionen zu visualisieren.

Um die relevantesten Chungs bereitzustellen, können wir das Embedding der Frage und der Chungs des Interviews nutzen, um die Ähnlichkeit zwischen ihnen zu berechnen. Anschließend können wir die Chunks mit der höchsten Ähnlichkeit zur Frage auswählen, um sie dem Modell zu geben.

Lass uns voran ein Embedding für eine beliebige Frage erzeugen:

In [55]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embedded_query = embeddings.embed_query("Wer ist Ralfs Bruder?")

print(f"Embedding Länge: {len(embedded_query)}")
print(embedded_query[:10])

Embedding Länge: 1536
[0.01599571667611599, -0.019886909052729607, 0.01371423527598381, -0.013169214129447937, -0.027580568566918373, 0.014462053775787354, -0.013625510968267918, -0.0007193002384155989, 0.0008008948643691838, -0.01744065433740616]


1536 beudetet in diesem Fall, dass das verwendete Embedding-Modell Text in einen 1536-dimensionalen Raum repräsentiert.

Um zu illustrieren, wie Embeddings funktionieren, generieren wir zunächst Embeddings für zwei verschiedene Sätze.

In [56]:
sentence1 = embeddings.embed_query("Ralfs Bruder ist Axl")
sentence2 = embeddings.embed_query("Michaelas Bruder ist Barbara")

Wir können jetzt die Ähnlichkeit zwischen der Query (Wer ist Ralfs Bruder?) und diesen beiden Sätzen berechnen. Je näher die Embeddings beieinander sind, desto ähnlicher sind sich die Query und der entsprechende Satz.

Wir nutzen die Cosine Similarity.

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9093964737341363, 0.8186437442297532)

Wir sehen, dass die Ähnlichkeit des Satzes "Ralfs Bruder ist Axl" zum Satz "Wer ist Ralfs Bruder?" sehr hoch ist mit 0,9 (der Score bewegt sich zwischen -1 und 1), was auch Sinn ergibt. Die Ähnlichkeit zu "Michaelas Schwester ist Barbara" zum Satz "Wer ist Ralfs Bruder?" ist ebenfalls hoch, weil alle 3 Sätze etwas mit Geschwisterverhältnissen zu tun haben. Allerdings hat der sentence1 zusätzlich noch direkt etwas mit den beteiligten Personen der query zu tun, weshalb dessen Wert noch etwas höher ist.

Wir brauchen jetzt einen effizienten Weg um die Chunks, deren Embeddings und deren Ähnlichkeits-Scores zu speichern. Dafür bedienen wir uns einer sog. Vektordatenbank. Eine Vektordatenbank ist eine Datenbank, die auf schnelle Ähnlichkeitssuchen spezialisiert ist.

Um zu verstehen wie eine Vektordatenbank funktioniert, erstellen wir eine in-memory and fügen ihr ein paar Embeddings hinzu:

In [58]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Ralfs Bruder ist Axl",
        "Michaelas Schwester ist Barbara",
        "Simon mag weiße Autos",
        "Janniks Vater ist Anlagenbauer",
        "Tassilo fährt einen Seat",
        "Sara hat eine Schwester",
    ],
    embedding=embeddings,
)

Wir können jetzt die Vektordatenbank befragen um die ähnlichsten Embeddings zu finden, gegeben der Query, die wir eingeben.

In [59]:
vectorstore1.similarity_search_with_score(query="Wer ist Ralfs Bruder?", k=3)

[(Document(metadata={}, page_content='Ralfs Bruder ist Axl'),
  0.9093964827944877),
 (Document(metadata={}, page_content='Michaelas Schwester ist Barbara'),
  0.8172974263814057),
 (Document(metadata={}, page_content='Sara hat eine Schwester'),
  0.8146215355810805)]

Wir sehen, dass die 3 Sätze, die am meisten mit Geschwisterverhältnissen zu tun haben, zurückgeliefert werden.

Wir können die Vektordatenbank benutzen, um die relevantesten Chunks des Interviews (gegeben einer bestimmten Frage) zu finden, um sie dem Modell zu übergeben. Wir können übrigens auch die Vektordatenbank mit einer Kette verbinden.

Wir müssen einen Retriever konfigurieren. Der Retriever wird eine Ähnlichkeitssuche in der Vektordatenbank durchführen und die relevantesten Dokumente an den nächsten Schritt in der Kette weitergeben. Den Retriever können wir direkt von der Vektordatenbank bekommen, die wir zuvor erstellt haben.

In [60]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Wer ist Ralfs Bruder?")

[Document(metadata={}, page_content='Ralfs Bruder ist Axl'),
 Document(metadata={}, page_content='Michaelas Schwester ist Barbara'),
 Document(metadata={}, page_content='Sara hat eine Schwester'),
 Document(metadata={}, page_content='Janniks Vater ist Anlagenbauer')]

Unser Prompt erwartet zwei Parameter, "context" und "question". Wir können den Retriever benutzen um die Chunks zu finden, die wir als den Kontext benutzen werden, um die Frage zu beantworten.

Wir können hierzu eine map mit zwei Inputs erstellen, indem wir RunnableParallel und RunnablePassthrough nutzen. Dies erlaubt uns den Kontext und die Frage an den Prompt zu übergeben als eine map mit den keys "context" und "question".

In [61]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("Welches Auto fährt Tassilo?")

{'context': [Document(metadata={}, page_content='Tassilo fährt einen Seat'),
  Document(metadata={}, page_content='Simon mag weiße Autos'),
  Document(metadata={}, page_content='Michaelas Schwester ist Barbara'),
  Document(metadata={}, page_content='Ralfs Bruder ist Axl')],
 'question': 'Welches Auto fährt Tassilo?'}

Jetzt können wir die map der Kette hinzufügen und laufen lassen.



In [62]:
chain = setup | prompt | model | parser
chain.invoke("Welches Auto fährt Tassilo?")

'Tassilo fährt einen Seat.'

Wir können die Kette auch mit einer anderen Query aufrufen.

In [63]:
chain.invoke("Was ist Janniks Vater von Beruf?")

'Janniks Vater ist Anlagenbauer.'

Oben haben wir die Vektordatenbank mit beliebigen Strings befüllt. Jetzt erzeugen wir eine neue Vektordatenbank für die Chunks des Interviews.

In [64]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

Wir können jetzt eine neue Kette erzeugen, indem wir die neue Vektordatenbank benutzen. Dieses Mal verwenden wir eine andere, aber äquivalente Syntax um den RunnableParallel-Teil der Kette zu spezifizieren.

In [65]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence refers to artificial intelligences that are seen as the next stage of development in technology. They are capable of learning, processing information, and potentially uncovering complex puzzles about the universe. These intelligences can generate content, engage in conversations about emotions, and exhibit reasoning by manipulating information and making predictions based on inputs.'